In [1]:
import random, struct,math
import argparse

In [2]:
BLACK	= '\033[30m'
RED		= '\033[31m'
GREEN	= '\033[32m'
YELLOW	= '\033[33m'
BLUE	= '\033[34m'
MAGENTA	= '\033[35m'
CYAN	= '\033[36m'
RESET	= '\033[0m'
SEL		= '\033[7m'

In [3]:
parser = argparse.ArgumentParser(description='fixed_mac')
parser.add_argument('--full_bits', type=int, default=16, help='Number of Quantization Bits')
parser.add_argument('--frac_bits', type=int, default=4, help='Number of Quantization Bits')
parser.add_argument('--bBW', type=int, default=4, help='Number of bit width')
args = parser.parse_args(args=[])


## Integer -> Two's Complemenet Binary

In [4]:
def int2bin(iIn,iBW):
    iBW = iBW + 1
    if iIn >= 0:
        bOut = bin(iIn).replace('0b','').rjust(iBW,'0')
    else :
        bOut = bin(iIn & (pow(2,iBW)-1)).replace('0b','').rjust(iBW,'1')
    return bOut[1:]

## XOR gate

In [5]:
def XOR(iA,iB):
    if iA != iB :
        iOut = '1'
    else : 
        iOut = '0'
    return iOut

## LFSR funnction

In [6]:
def olfsr(seed,flag,taps):
    sr, xor = seed, 0
    for t in taps :
        xor += int(sr[t-1])
        #print(xor)
    if xor%2 == 0:
        xor = 0
    else :
        xor = 1
    sr, xor = str(xor) + sr[:-1], 0
    num = int(sr,2)
    if sr == flag[0]:
        return len(seed)*"0"
    else :
        return sr

### Comparator

In [7]:
def Comp(a,lfsr):
    for com in range(0,len(lfsr)):
        oA = '0'
        if a[com+1]!=lfsr[com]:
            if(int(a[com+1]) > int(lfsr[com])):
                oA = '1'
            break
    return XOR(oA,a[0])

### lfsr shifter

k bit shifter : lfsr의 bit를 kbit만큼 shift 해서 자리 바꿔줌, 이 때 k는 lfsr 길이의 절반

In [8]:
def kshift(a,k):
    if type(a) != str :
        print("input is not string type")
        return 0
    b = (a[k:]+ a[:k])
    return b

## SNG

In [9]:
def SNG(a,b,aBW,bBW):
    #iA = int2bin(a,bBW)
    #iB = int2bin(b,bBW)
    
    #sNUM = iB[0] #signed number
    
    oAlist = []
    oBlist = []
    lfsrlist = []
    ran = random.randint(1,math.pow(2,args.bBW)-1)
    ranZP = args.bBW-len(bin(ran)[2:])
    num = ranZP*'0'+bin(ran)[2:] #lfsr start number generate
    for k in range(2**(args.bBW)): #lfsr number generating
        if not lfsrlist:
            lfsrlist.append(num)
        else :
            lfsrlist.append(olfsr(lfsrlist[k-1],lfsrlist,(0,1)))
        alNUM = lfsrlist[k] #lfsr of input a --> number generating
        blNUM = kshift(alNUM,len(alNUM)//2) #lfsr of input b --> k bit shifting
        
        oAlist.append(Comp(a[0]+a[len(a)-args.frac_bits-aBW:len(a)-args.frac_bits],alNUM)) #comparator of input a
        oBlist.append(Comp(b[0]+b[len(b)-args.frac_bits-bBW:len(b)-args.frac_bits],blNUM)) #comparator of input b
    
    oAnum = oAlist.count('1')
    oBnum = oBlist.count('1')
    
    
    
    #if (oAnum != a) or (oBnum != b) :
    #    print("SNG is fail")
    #    return 0
    
    
    oAlist.insert(0,a[0])
    oBlist.insert(0,b[0])
    sA = "".join(oAlist)
    sB = "".join(oBlist)
    return sA,sB

In [10]:
s = "00000110100000000"
print(s[len(s)-args.frac_bits-args.bBW:len(s)-args.frac_bits])


0000


## MAC

### mult

In [11]:
def mul(a,b):
    al = len(a)
    bl = len(b)
    
    outlist = []
    
    if al != bl :
        print("length of string is different")
        return 0
    
    for i in range(al) :
        outlist.append(str(int(a[i]) & int(b[i])))
    
    #print(outlist)
    out = "".join(outlist)
    
    return out    

In [ ]:
def count1(sto):
    n = 0
    for i in range(len(sto)):
        if sto[i] == '1' :
                   n += 1
    
    return n

# Fixed point test

## fixed point function

In [13]:
class	fxp:
	def	__init__(self, bIn, iBWF):
		self.iFullBW	= len(bIn)
		self.iIntgBW	= self.iFullBW - iBWF
		self.bSign		= bIn[0]
		self.bIntg		= bIn[:self.iIntgBW]
		self.bFrac		= bIn[self.iIntgBW:]
		self.fFull		= 0
		try:
			for idx, bit in enumerate(bIn):
				if	idx == 0:
					self.fFull = self.fFull + int(bit,2) * -pow(2, self.iIntgBW - 1)
				else:
					self.fFull = self.fFull + int(bit,2) * pow(2, self.iIntgBW - 1 - idx)
		except:
			print(bIn)
		self.dispFull	= RED + self.bIntg + BLUE + self.bFrac + RESET
		return

In [14]:
class	flp2fix:
	def	__init__(self, fIn, iBW, iBWF):
		self.fMin		= - 2 ** (iBW - iBWF - 1)
		self.fMax		= (2 ** (iBW-1) - 1) * (2 ** -iBWF)
		self.fResol		= 2 ** -iBWF
		if fIn < self.fMin or fIn > self.fMax:
			print(f'({fIn}): Out of input range ({self.fMax}/{self.fMin}) during flp -> fix converting ')
		self.iBW		= iBW
		self.iBWI		= iBW - iBWF
		self.iBWF		= iBWF

		self.iFLP2INT	= abs(int(fIn * 2 ** iBWF))
		if fIn < 0:
			self.iFLP2INT = 2 ** (iBW-1) - self.iFLP2INT

		if fIn >= 0:
			self.bFull = bin(self.iFLP2INT)[2:].rjust(iBW, '0')
		else:
			self.bFull = '1'+bin(self.iFLP2INT)[2:].rjust(iBW-1, '0')
			if len(self.bFull) > iBW:
				self.bFull = '0' * iBW

		self.cssFxp		= fxp(self.bFull, self.iBWF)
		self.bSign		= self.cssFxp.bSign
		self.bIntg		= self.cssFxp.bIntg
		self.bFrac		= self.cssFxp.bFrac
		self.fFull		= self.cssFxp.fFull
		return

## fixe point test

In [16]:
n = 0
for i in range (10000) :

    Ra = random.randint(0,2**(args.bBW)-1)
    Rb = random.randint(0,2**(args.bBW)-1)
    Rresult = Ra * Rb
    
    aBW = len(int2bin(int(Ra),args.bBW))
    bBW = len(int2bin(int(Rb),args.bBW))
    rBW = len(int2bin(int(Rresult),args.bBW))
    
    
    Fa = flp2fix(Ra,args.full_bits, args.frac_bits).bSign + flp2fix(Ra,args.full_bits, args.frac_bits).bFull
    Fb = flp2fix(Rb,args.full_bits, args.frac_bits).bSign + flp2fix(Rb,args.full_bits, args.frac_bits).bFull
    Fresult = flp2fix(Rresult,args.full_bits,args.frac_bits).fFull
    
    Sa,Sb = SNG(Fa,Fb,aBW,bBW) 
    
    Sout = mul(Sa,Sb)
    
    Sresult = Sout[1:]*len(Sa[1:])
    
    NumOne = count1(Sresult)
 
    FNumOne = flp2fix(NumOne,args.full_bits,args.frac_bits).fFull
    
    ResultValue = flp2fix(Rresult,args.full_bits,args.frac_bits).fFull/pow(2**(args.bBW),2)
    StoValue = FNumOne/pow(2**(args.bBW),2)
    
    percent = abs(StoValue-ResultValue)*100
    if (100-percent) > 95.0 :
        n += 1
    
    #print(f'Ra : {Ra:>30n},{Sa}')
    #print(f'Rb : {Rb:>30n},{Sb}')
    
    #print(f'result : {Rresult:>26n}')
    #print(f'multiplier stohastic number is : {Sout:>20s}')
    #print(f'NumOne is : {FNumOne:>23n}')
    #print(f'ResultValue is : {ResultValue:>25f}')
    #print(f'StoValue is : {StoValue:>28f}')
    #print(f'percentage is : {100-percent:>26f}')
    #print(f'----------------------------------------------------------')     


In [17]:
print(f' Succes is : {n}/{10000}')

 Succes is : 6562/10000
